In [11]:
import pandas as pd
import numpy as np
from urllib.request import urlopen #url의 html 을 가져 오기 위한 패키지
from bs4 import BeautifulSoup  #크롤링 필수 패키지 설치하려면 cmd창에서 pip install bs4
import os
import re
from selenium import webdriver
from bs4 import BeautifulSoup #크롤링 도구
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import os
import re
import time
import json
import datetime
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pylab as pl
import statsmodels.api as sm
from sklearn.cluster import DBSCAN
import sklearn as skl
import sklearn.model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from ckonlpy.tag import Twitter
import string
import glob
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import warnings




twi = Twitter()
okt = Okt()
twi.nouns('확진자와 비대면으로 거리두기를 사랑제일교회에서부터 전광훈이 사전조사 의료기관과 의료진이 집단감염이 걸려 집단감염, 재택근무하고 '
          '유연근무와 유연근무제를 하고 지역경제와 한국판뉴딜과 그린뉴딜과 디지털뉴딜을 휴면뉴딜하고 취약계층의 긴급재난지원금을 특수고용직에게'
          '고용보험을 진료거부하며 택배기사는 온라인수업으로 선별진료소로 간다') #확인용
okt.nouns('확진자와 비대면으로 거리두기를 사랑제일교회에서부터 전광훈이 사전조사 의료기관과 의료진이 집단감염이 걸려 집단감염, 재택근무하고 '
          '유연근무와 유연근무제를 하고 지역경제와 한국판뉴딜과 그린뉴딜과 디지털뉴딜을 휴면뉴딜하고 취약계층의 긴급재난지원금을 특수고용직에게'
          '고용보험을 진료거부하며 택배기사는 온라인수업으로 선별진료소로 간다') #확인용
#단어 추가 하기      '서구 문학'일 경우는 서구 와 문학으로 나뉨 띄어쓰기가 안됨
twi.add_dictionary('비대면','Noun')
twi.add_dictionary('확진자','Noun')
twi.add_dictionary('거리두기','Noun')
twi.add_dictionary('사랑제일교회','Noun')
twi.add_dictionary('의료기관','Noun')
twi.add_dictionary('의료진','Noun')
twi.add_dictionary('집단감염','Noun')
twi.add_dictionary('유연근무','Noun')
twi.add_dictionary('한국판뉴딜','Noun')
twi.add_dictionary('그린뉴딜','Noun')
twi.add_dictionary('디지털뉴딜','Noun')
twi.add_dictionary('휴면뉴딜','Noun')
twi.add_dictionary('취약계층','Noun')
twi.add_dictionary('긴급재난지원금','Noun')
twi.add_dictionary('고용보험','Noun')
twi.add_dictionary('진료거부','Noun')
twi.add_dictionary('택배기사','Noun')
twi.add_dictionary('선별진료소','Noun')
twi.add_dictionary('고용보험','Noun')
twi.add_dictionary('특수고용직','Noun')


data = pd.read_excel('C:/Users/USER/Desktop/1~8newsurl/covid_url38_word50.xlsx', index_col=0)

list_text = data['text']
words = [twi.nouns(i) for i in tqdm(list(list_text))] #시간 걸림

Prohibit_words = ['기자','연합뉴스','뉴시스','시사저널','신문','뉴스','사진','헤럴드경제','노컷뉴스','파이낸셜뉴스','특파원',
                  '라며','대해','지난','위해','오전','오후','무단','배포','이데일리','머니투데이','앵커','지금','때문','이번',
                  '통해','정도','경우','관련','이미지','출처','일보']

j = 0
for i in tqdm(words):
    for k in Prohibit_words:
        while k in i:
            i.remove(k)
    words[j] = i
    j += 1 #불용어 제외

for k in range(len(words)):
    words[k] = [i for i in words[k] if len(i) > 1]  # 한글자 제외

data['words'] = words

data['length_word'] = [len(i) for i in tqdm(data['words'])]

data50 = data[data['length_word'] >= 50]  #단어 갯수 50개 이하 제거

embedding_model = Word2Vec(list(data50['words']), size=100, window = 6, min_count=150, workers=4, iter=100)
embedding_model.wv.save_word2vec_format('C:/Users/USER/Desktop/백업/covid_model38_2')# 모델 저장
data50.to_excel('C:/Users/USER/Desktop/1~8newsurl/covid_url38_word50_2.xlsx',header=True)
news = data50['words'].tolist()
#news = [str(sent).split(',') for sent in news]
print(len(news))
print(str(news[0]))
#사전 만들기
id2word = corpora.Dictionary(news)

texts = news

corpus = [id2word.doc2bow(text) for text in texts]

# https://dacon.io/competitions/official/235590/codeshare/974
#토픽모델링 함수
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
      print(num_topics)
      model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                              id2word=id2word,
                                              num_topics=num_topics,
                                              random_state=100,
                                              update_every=1,
                                              chunksize=100,
                                              passes=10,
                                              alpha='auto',
                                              per_word_topics=True)
      model_list.append(model)
      coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
      coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=31, step=2)

# Show graph
limit=31; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()


# Print the coherence scores
for m, cv in zip(x, coherence_values):
  print("Num Topics =", m, " has Coherence Value of", round(cv, 4))


# Select the model and print the topics
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=15))



C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')

  1%|▍                                                                             | 381/63661 [00:08<21:40, 48.67it/s]


  1%|▌                                                                             | 446/63661 [00:10<26:11, 40.23it/s]

KeyboardInterrupt: 

In [2]:
# Select the model and print the topics
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=15))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.209925  0.181865       1        1  34.103714
4      0.123108 -0.127751       2        1  16.669579
11     0.129961 -0.145050       3        1   9.898994
7      0.110391 -0.093829       4        1   7.561217
1      0.068012  0.012648       5        1   7.432616
6      0.075122  0.016355       6        1   6.293575
3     -0.037948 -0.013433       7        1   5.685457
12     0.068944  0.181999       8        1   4.822873
2     -0.060523 -0.074790       9        1   3.355181
13    -0.076808 -0.018573      10        1   1.909283
9     -0.087589 -0.002873      11        1   1.029987
5     -0.184779  0.031086      12        1   0.504333
0     -0.169757  0.030827      13        1   0.425198
10    -0.168057  0.021517      14        1   0.307997, topic_info=        Term        Freq       Total Category  logprob  loglift
626    '마스크'  176.000000  176.000000  Default  30.0000  30.0000
442    '확진자'  462.000000  462.000000  Default  29.0000  29.0000
481     '수출'  157.000000  157.000000  Default  28.0000  28.0000
290    '신천지'  228.000000  228.000000  Default  27.0000  27.0000
9       '방역'  162.000000  162.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
2580    '특수'    0.391667    5.871680  Topic14  -5.7244   3.0754
1542    '사용'    0.587884   22.920860  Topic14  -5.3182   2.1196
2561    '신장'    0.331341    5.911520  Topic14  -5.8916   2.9013
461     '대비'    0.226390   58.617683  Topic14  -6.2725   0.2263
1637    '거실'    0.211918    1.731924  Topic14  -6.3386   3.6820

[810 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
446       2  0.571239    '가격'
446       3  0.244817    '가격'
446       4  0.020401    '가격'
446       8  0.102007    '가격'
446       9  0.040803    '가격'
...     ...       ...     ...
2759      1  0.333676    '휴일'
2759     13  0.333676    '휴일'
1756      8  0.866079    '휴지'
616       3  0.953277    '흑자'
905      10  0.515174  ['박원순'

[1581 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 12, 8, 2, 7, 4, 13, 3, 14, 10, 6, 1, 11])